# Twitter sentiment analysis

## Load the data

Please, load data from https://inclass.kaggle.com/c/si650winter11

In [1]:
%env THEANO_FLAGS='floatX=float32'

env: THEANO_FLAGS='floatX=float32'


In [2]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
data = []
labels = []
with open('./training.txt') as f:
    for line in f:
        tokens = line.split('\t')
        label, tweet = tokens[0], tokens[1:]
        labels.append(int(label))
        data.append('\t'.join(tweet))

In [ ]:
labels = np.array(labels)

Synonyms were obtained by searching for top 20 most similar words accoroding to Glove twitter model: https://nlp.stanford.edu/projects/glove/

In [3]:
import json

with open('synonyms.json') as f:
    synonyms = json.load(f)

For each word a list with at most 20 words is provided. Along with synonyms similarity coefficients are given (the large coefficient the more similar the synonym is).

In [4]:
synonyms['man']

[['dude', 0.7253455519676208],
 ['boy', 0.7174534797668457],
 ['guy', 0.6998012065887451],
 ['shit', 0.6852256655693054],
 ['was', 0.6779443621635437],
 ["'s", 0.6762615442276001],
 ['bad', 0.6734011769294739],
 ['men', 0.6714873909950256],
 ['that', 0.6709873080253601],
 ['lol', 0.6646672487258911],
 ['fuck', 0.6629763841629028],
 ['woman', 0.6613606214523315],
 ['but', 0.6589959263801575],
 ['bro', 0.6560863256454468],
 ['way', 0.6556506752967834],
 ['girl', 0.6554247736930847],
 ['nigga', 0.6549790501594543],
 ['who', 0.6531668305397034],
 ['just', 0.6512974500656128],
 ['so', 0.6502212882041931]]

### Getting all words used in the dataset

Here you can do some feature selection e.g. by word frequency or by tf-idf.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

vectorizer.fit(data)
vocab = list(vectorizer.vocabulary_.keys())

### Select some informative words

In [ ]:
### change me!
pruned_vocab = vocab[:100]

### Extend them with synonyms

In [ ]:
### CHANGE ME
extended_vocab = pruned_vocab + [ synonyms[w][0][0] for w in pruned_vocab if w in synonyms ]

In [ ]:
extended_vocab_set = set(extended_vocab)

### to exclude repeating words
extended_vocab = list(extended_vocab_set)

In [ ]:
word_to_index = dict(zip(
    extended_vocab,
    range(len(extended_vocab))
))

index_to_word = dict(zip(
    range(len(extended_vocab)),
    extended_vocab,
))

In [ ]:
vectorizer = CountVectorizer(vocabulary=extended_vocab)

In [ ]:
X = vectorizer.transform(data).toarray()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.25)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

### Computing links between words

In [ ]:
word_ids1 = []
word_ids2 = []

for w1 in extended_vocab:
    if w1 not in synonyms:
        continue

    for w2, _ in synonyms[w1]:
        if w2 in extended_vocab_set:
            word_ids1.append(word_to_index[w1])
            word_ids2.append(word_to_index[w2])

In [ ]:
word_ids1 = np.array(word_ids1)
word_ids2 = np.array(word_ids2)

In [ ]:
print(word_ids1)
print(word_ids2)

### Implementing logistic regression

If ypu don't have theano or lasagne, please, follow

http://lasagne.readthedocs.io/en/latest/user/installation.html#bleeding-edge-version

```
pip install --upgrade https://github.com/Theano/Theano/archive/master.zip
pip install --upgrade https://github.com/Lasagne/Lasagne/archive/master.zip
```

In [ ]:
import theano
import theano.tensor as T

In [ ]:
X = T.fmatrix()
y = T.fvector()

In [ ]:
n_features = len(extended_vocab)

W = theano.shared(
    ### don't do such initialization in neural networks!!!
    np.random.uniform(-1.0e-2, 1e-2, size=(n_features, 1)).astype('float32')
)

b = theano.shared(np.float32(0.0))

link_words_1 = theano.shared(word_ids1.astype('int64'))
link_words_2 = theano.shared(word_ids2.astype('int64'))

In [ ]:
predictions = T.flatten(
    T.nnet.sigmoid(T.dot(X, W) + b),
    ndim=1
)

In [ ]:
cross_entropy_loss = -T.mean(
    y * T.log(predictions) + (1 - y) * T.log(1 - predictions)
)

In [ ]:
l2_regularisation = 1.0e-2 * T.sum(W ** 2)

In [ ]:
l2_regularisation.eval()

#### And ... our exotic regularization

In [ ]:
### Change me!
synonim_regularization = 1.0e-2 * T.sum(W ** 2)

In [ ]:
synonim_regularization.eval()

In [ ]:
### Adjust my coefficients!
loss = cross_entropy_loss + l2_regularisation + synonim_regularization

Training part.

If you don't have lasagne just copy adamax optimizer from:


In [ ]:
from lasagne.updates import adamax

In [ ]:
train = theano.function(
    inputs=[X, y],
    outputs=[cross_entropy_loss, l2_regularisation, synonim_regularization],
    updates=adamax(loss, [W, b])
)

In [ ]:
def random_seq(n_samples, batch_size=128, allow_smaller=False):
  indx = np.random.permutation(n_samples)

  n_batches = n_samples // batch_size + (1 if (n_samples % batch_size != 0) and allow_smaller else 0)

  for i in range(n_batches):
    i_from = i * batch_size
    i_to = i_from + batch_size
    yield indx[i_from:i_to]

In [ ]:
n_epoches = 16
batch_size = 8
n_batches = X_train.shape[0] // batch_size

losses = np.ndarray(shape=(n_epoches, n_batches), dtype='float32')
l2_reg = np.ndarray(shape=(n_epoches, n_batches), dtype='float32')
synonym_reg = np.ndarray(shape=(n_epoches, n_batches), dtype='float32')

In [ ]:
print(X_train.shape)

In [ ]:
from nn_watcher import SNNWatcher

watcher = SNNWatcher(
    title='LogReg-tweets',
    colors=('blue', 'green', 'red'),
    labels=('cross-entropy loss', 'l2', 'synonym'),
    mode='fill',
)

for i in range(n_epoches):
    for j, indx in enumerate(random_seq(X_train.shape[0], batch_size=batch_size)):
        losses[i, j], l2_reg[i, j], synonym_reg[i, j] = train(
            X_train[indx].astype('float32'),
            y_train[indx].astype('float32')
        )
    watcher.draw(
        losses[:(i + 1)],
        l2_reg[:(i + 1)],
        synonym_reg[:(i + 1)],
    )

## Test it

- test your initial vocab;
- test your extended vocab without fancy regularization;
- test your extended vocab with fancy regularization;
- adjust regularization coefficients to obtain best score (don't forget to make train/val/test split).